# Despeckle script

- This notebook preprocesses raw sentinel-1 data with a magic transform (see report) and despeckles it with the SAR2SAR algorithm.
- The SAR2SAR script we used can be found here (https://gitlab.telecom-paris.fr/ring/sar2sar/-/tree/master/), it is not included in this repository for license reasons (use GRD version)

## To run the script
- Select year and resolution in cell below and call despeckle() function to run the script
- Denoising a single eo-patch takes about 15 minutes
- The results are stored in despeckle_{chosen_year} with the same folder structure as the data downloaded from sentinelhub is
- Both a Single threaded and a multicore version was created, the single threaded version is in general more stable but really slow, the parallelized version is fast but crashes frequently and need constant maintenance.
- The results will appear in despeckle_sar2sarV3_res_11m (with SELECTED

In [1]:
SELECTED_YEARS = [2019]
SELECTED_RES = 11
WHOLE_YEAR = False #Whole year was added so we could put these files in a different folder

In [2]:
import os
import glob
import numpy as np
import re
import matplotlib.pyplot as plt
import shutil
import concurrent.futures

import subprocess

from util_paths import SENTINEL_1_11x11_PATH

In [3]:
#Cell for cleaning tmp data
for path in glob.glob(os.path.join(os.getcwd(), "despeckle_tmp*")):
    shutil.rmtree(path)

In [4]:
#Find folder for selected years and resolution
SENT_1_MAIN_FOLDER = SENTINEL_1_11x11_PATH
SENT_1_YEAR_FOLDERS = {}
DESPECKLE_FINAL_FOLDERS = {}
if not WHOLE_YEAR:
    DESPECKLE_FINAL_RES_FOLDER = os.path.join(os.getcwd(), f"despeckle_sar2sarV3_res_{SELECTED_RES}m")

    for year in SELECTED_YEARS:
        SENT_1_YEAR_FOLDERS[year] = os.path.join(SENT_1_MAIN_FOLDER, f"res_{SELECTED_RES}m", f"{year}")
        DESPECKLE_FINAL_FOLDERS[year] = os.path.join(DESPECKLE_FINAL_RES_FOLDER, f"{year}")

    DESPECKLE_TMP_FOLDER = os.path.join(os.getcwd(), "despeckle_tmp")
    DESPECKLE_TMP_RESULTS_FOLDER = os.path.join(os.getcwd(), "despeckle_tmp_results")
if WHOLE_YEAR:
    DESPECKLE_FINAL_RES_FOLDER = os.path.join(os.getcwd(), f"despeckle_sar2sarV3_whole_year_res_{SELECTED_RES}m")
    for year in SELECTED_YEARS:
        SENT_1_YEAR_FOLDERS[year] = os.path.join(SENT_1_MAIN_FOLDER,'whole_year',f"{year}")
        DESPECKLE_FINAL_FOLDERS[year] = os.path.join(DESPECKLE_FINAL_RES_FOLDER, f"{year}")
    DESPECKLE_TMP_FOLDER = os.path.join(os.getcwd(), "despeckle_tmp")
    DESPECKLE_TMP_RESULTS_FOLDER = os.path.join(os.getcwd(), "despeckle_tmp_results")

In [5]:
#The treshholding and log transform happens here
# - matrix: 2D matrix for a single time stamp and chosen polarization (0 or 1)
def old_preprocess_log_tresh(matrix, polarization):
    #matrix[matrix>1] = 1                           # Treshhold
    if polarization == 0:                           # Decibel transform with magic numbers which gave good results
        matrix = np.abs( 40*np.log10(0.5*matrix) )  # The idea behind the numbers is that they introduce some more dynamic range between 0 to 255
    else:                                           # which the model seems to like
        matrix = np.abs( 40*np.log10(4*matrix) )
    matrix[np.isinf(matrix)] = 255                  # Remove inf values (this is a result of using log on 0)
    matrix[matrix>255] = 255                        # Second treshhold since model seem to be wanting numbers between 0 to 255
    return matrix

def preprocess_transform(matrix, polarization):
    if polarization == 0:                                      
        log_base = 1.8                                        # Choose log base which gives good denoising results
        matrix =  20*np.log(50*2*matrix + 2)/np.log(log_base) # Compute log transform and do final scaling to match sar2sar examples
    else:                                         
        log_base = 1.8
        matrix =  20*np.log(50*10*matrix + 2)/np.log(log_base) 
    matrix[np.isinf(matrix)] = 100
    return matrix

In [22]:
#Given a path to a patch and where to store despeckling results
# - extract raw data into single numpy matrices in despeckle_tmp folder
# - preprocess by logtransform and tresholding
# - use sar2sar script and put results in despeckle_tmp_results folder
# - reconstruct the radar tensor and save in final_result_path
# - remove all tmp data
def despeckle_eo_patch(eo_patch_raw_matrix_path, final_matrix_path, sar2sar_stride = 32):
    
    #Make sure required folders exist
    if not os.path.exists(DESPECKLE_TMP_FOLDER):
        os.mkdir(DESPECKLE_TMP_FOLDER)
        
    if not os.path.exists(DESPECKLE_TMP_RESULTS_FOLDER):
        os.mkdir(DESPECKLE_TMP_RESULTS_FOLDER)
    
    #Extract raw data into despeckle_tmp folder
    raw_radar_tensor = np.load(eo_patch_raw_matrix_path)
    radar_tensor_shape = raw_radar_tensor.shape
    n_timepoints = raw_radar_tensor.shape[0]
    n_polarizations = raw_radar_tensor.shape[3]
    for t in range(n_timepoints):
        for p in range(n_polarizations):
            extracted_matrix = preprocess_transform(raw_radar_tensor[t,:,:,p], p)
            extracted_matrix_path = os.path.join(DESPECKLE_TMP_FOLDER, f"t{t}_p{p}.npy")
            np.save(extracted_matrix_path, extracted_matrix)
    
    #Run sar2sar script
    !python sar2sar_tf2/content/SAR2SAR-GRD-test/main.py --test_data {DESPECKLE_TMP_FOLDER} --test_dir {DESPECKLE_TMP_RESULTS_FOLDER} --stride_size={sar2sar_stride}
    
    #Reconstruct original radar tensor
    denoised_radar_tensor = np.zeros(shape = radar_tensor_shape)
    denoised_matrix_paths = glob.glob(os.path.join(DESPECKLE_TMP_RESULTS_FOLDER, "*"))
    pattern = r"denoised_t(\d+)_p(\d).npy"
    for path in denoised_matrix_paths:
        t, p = re.search(pattern, path).groups()
        t = int(t)
        p = int(p)
        denoised_radar_tensor[t,:,:,p] = np.load(path)
    
    np.save(final_matrix_path, denoised_radar_tensor)
    
    #Remove all tmp files after reconstruction
    for path in glob.glob(os.path.join(DESPECKLE_TMP_FOLDER, "*")):
        os.remove(path)
    for path in denoised_matrix_paths:
        os.remove(path)
    
    

In [7]:
#Given a year path, find all corresponding eo-patches paths, decide where to put the results and despeckle them
def despeckle_year(year, sar2sar_stride = 32, verbose = False):
    eo_patch_paths = glob.glob(SENT_1_YEAR_FOLDERS[year] + '/*')
    print(f"########## Despeckling year {year} #############")
    for eo_patch_path in eo_patch_paths:
        #Extract patch id and find path of the raw matrix
        eo_patch_id = int(re.search(r'.eopatch_(\d+)', eo_patch_path).group(1))
        eo_patch_raw_matrix_path = os.path.join( eo_patch_path , 'data' , 'IW.npy')
        
        #Construct all neccessary folder structure for a single year
        final_eo_patch_path = os.path.join(DESPECKLE_FINAL_FOLDERS[year], f"eopatch_{eo_patch_id}")
        final_eo_patch_data_path = os.path.join(final_eo_patch_path, "data")
        final_matrix_path = os.path.join( final_eo_patch_data_path, "IW.npy")
        
        if not os.path.exists(final_eo_patch_path):
            os.mkdir(final_eo_patch_path)
        if not os.path.exists(final_eo_patch_data_path):
            os.mkdir(final_eo_patch_data_path)
        
        #Help prints for debugging this function
        if verbose:
            print(eo_patch_id)
            print(eo_patch_path)
            print(eo_patch_raw_matrix_path) 
            print()
            print(final_eo_patch_path)
            print(final_eo_patch_data_path)
            print(final_matrix_path)
            
        print(f"########## Despeckling eo-patch {eo_patch_id} #########")
            
        #Copy metadata from raw data folder into despeckled folder
        shutil.copy2(os.path.join(eo_patch_path, "bbox.geojson"), os.path.join(final_eo_patch_path, "bbox.geojson"))
        shutil.copy2(os.path.join(eo_patch_path, "meta_info.json"), os.path.join(final_eo_patch_path, "meta_info.json"))
        shutil.copy2(os.path.join(eo_patch_path, "timestamp.json"), os.path.join(final_eo_patch_path, "timestamp.json"))
            
        despeckle_eo_patch(eo_patch_raw_matrix_path, final_matrix_path, sar2sar_stride=sar2sar_stride)

In [9]:
def despeckle(sar2sar_stride = 32):
    for year in SELECTED_YEARS:
        if not os.path.exists(DESPECKLE_FINAL_RES_FOLDER):
            os.mkdir(DESPECKLE_FINAL_RES_FOLDER)
        if not os.path.exists(DESPECKLE_FINAL_FOLDERS[year]):
            os.mkdir(DESPECKLE_FINAL_FOLDERS[2020])
        despeckle_year(year, sar2sar_stride=sar2sar_stride)

# Parallelizing 

### Second warning: This is extremely unstable and needs constant maintenance and restarts for unknown reasons

In [6]:
def handle_eo_patch_conc(args):
    
    #We check if the final matrix already exists, in which case we just ignore this job (This is to restart after crash or notebook shutdown)
    if os.path.exists(args["final_matrix_path"]):
        print(f"year {args['year']}, eopatch_{args['eo_patch_id']} already exist, skipping \n")
        return
    else:
        print(f"Despeckling year {args['year']}, eopatch {args['eo_patch_id']} \n")
    
    #Setup required folder structure
    if not os.path.exists(args["final_eo_patch_path"]):
        os.mkdir(args["final_eo_patch_path"])
    if not os.path.exists(args["final_eo_patch_data_path"]):
        os.mkdir(args["final_eo_patch_data_path"])
    if not os.path.exists(args["despeckle_tmp_folder"]):
        os.mkdir(args["despeckle_tmp_folder"])
    if not os.path.exists(args["despeckle_tmp_result_folder"]):
        os.mkdir(args["despeckle_tmp_result_folder"])
    
    for path in glob.glob(os.path.join(args["despeckle_tmp_folder"], "*")):
        os.remove(path)
    for path in glob.glob(os.path.join(args["despeckle_tmp_result_folder"], "*")):
        os.remove(path)
    
    #Copy metadata 
    shutil.copy2(os.path.join(args["eo_patch_path"], "bbox.geojson"), os.path.join(args["final_eo_patch_path"], "bbox.geojson"))
    shutil.copy2(os.path.join(args["eo_patch_path"], "meta_info.json"), os.path.join(args["final_eo_patch_path"], "meta_info.json"))
    shutil.copy2(os.path.join(args["eo_patch_path"], "timestamp.json"), os.path.join(args["final_eo_patch_path"], "timestamp.json"))
    
    #Extract raw data into despeckle_tmp folder
    raw_radar_tensor = np.load(args["eo_patch_raw_matrix_path"])
    radar_tensor_shape = raw_radar_tensor.shape
    n_timepoints = raw_radar_tensor.shape[0]
    n_polarizations = raw_radar_tensor.shape[3]
    for t in range(n_timepoints):
        for p in range(n_polarizations):
            extracted_matrix = preprocess_transform(raw_radar_tensor[t,:,:,p], p)
            extracted_matrix_path = os.path.join(args["despeckle_tmp_folder"], f"t{t}_p{p}.npy")
            np.save(extracted_matrix_path, extracted_matrix)
    
    #Run sar2sar script and wait for result
    !python sar2sar_tf2/content/SAR2SAR-GRD-test/main.py \
        --test_data {args["despeckle_tmp_folder"]} \
        --test_dir {args["despeckle_tmp_result_folder"]} \
        --stride_size={args["sar2sar_stride"]} \
        --use_gpu 1 \
        --gpu_memory_fraction {args['gpu_memory_fraction']} 
    
    
    #Reconstruct original radar tensor with denoised data
    denoised_radar_tensor = np.zeros(shape = radar_tensor_shape)
    denoised_matrix_paths = glob.glob(os.path.join(args["despeckle_tmp_result_folder"], "*"))
    pattern = r"denoised_t(\d+)_p(\d).npy"
    for path in denoised_matrix_paths:
        t, p = re.search(pattern, path).groups()
        t = int(t)
        p = int(p)
        denoised_radar_tensor[t,:,:,p] = np.load(path)
        
    np.save(args["final_matrix_path"], denoised_radar_tensor)
    
    #Remove all tmp files after reconstruction
    #for path in glob.glob(os.path.join(args["despeckle_tmp_folder"], "*")):
    #    os.remove(path)
    #for path in denoised_matrix_paths:
    #    os.remove(path)
    if os.path.exists(args["despeckle_tmp_folder"]):
        os.remove(args["despeckle_tmp_folder"])
    if os.path.exists(args["despeckle_tmp_result_folder"]):
        os.remove(args["despeckle_tmp_result_folder"])

In [7]:
def despeckle_conc(n_workers = 4, sar2sar_stride = 32):
    total_gpu_memory_usage = 0.9
    memory_per_worker = total_gpu_memory_usage/n_workers
    
    args_list = []
    for year in SELECTED_YEARS:
        if not os.path.exists(DESPECKLE_FINAL_RES_FOLDER):
            os.mkdir(DESPECKLE_FINAL_RES_FOLDER)
        if not os.path.exists(DESPECKLE_FINAL_FOLDERS[year]):
            os.mkdir(DESPECKLE_FINAL_FOLDERS[year])
        for eo_patch_path in glob.glob(os.path.join(SENT_1_YEAR_FOLDERS[year], "*")):
            
            eo_patch_id = int(re.search(r'.eopatch_(\d+)', eo_patch_path).group(1))
            
            args = {}
            args["year"] = year
            args["eo_patch_id"] = eo_patch_id
            args["eo_patch_path"] = eo_patch_path
            args["eo_patch_raw_matrix_path"] = os.path.join(eo_patch_path, "data", "IW.npy")
            args["final_eo_patch_path"] = os.path.join(DESPECKLE_FINAL_FOLDERS[year], f"eopatch_{eo_patch_id}")
            args["final_eo_patch_data_path"] = os.path.join(args["final_eo_patch_path"], "data")
            args["final_matrix_path"] = os.path.join(args["final_eo_patch_data_path"], "IW.npy")
            args["sar2sar_stride"] = sar2sar_stride
            
            args["despeckle_tmp_folder"] = os.path.join(os.getcwd(), f"despeckle_tmp_{year}_{eo_patch_id}")
            args["despeckle_tmp_result_folder"] = os.path.join(os.getcwd(), f"despeckle_tmp_result_{year}_{eo_patch_id}")
            args["gpu_memory_fraction"] = memory_per_worker
            
            args_list.append(args)
            
    with concurrent.futures.ProcessPoolExecutor(max_workers=n_workers) as executor:
        executor.map(handle_eo_patch_conc, args_list)
        #for args in args_list:
        #    print(f"Starting year: {args['year']}, eopatch: {args['eo_patch_id']}")
        #    executor.submit(handle_eo_patch_conc, args)
        #executor.shutdown(wait = True)

In [8]:
despeckle_conc(n_workers=4)

year 2019, eopatch_42 already exist, skipping 
year 2019, eopatch_43 already exist, skipping 
year 2019, eopatch_117 already exist, skipping 



Despeckling year 2019, eopatch 78 
year 2019, eopatch_79 already exist, skipping 


year 2019, eopatch_77 already exist, skipping 



<ipython-input-5-59a87a6b8a22>:19: RuntimeWarning: invalid value encountered in log
  matrix =  20*np.log(50*10*matrix + 2)/np.log(log_base)
<ipython-input-5-59a87a6b8a22>:16: RuntimeWarning: invalid value encountered in log
  matrix =  20*np.log(50*2*matrix + 2)/np.log(log_base) # Compute log transform and do final scaling to match sar2sar examples


2022-10-24 09:24:00.931553: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-10-24 09:24:07.356349: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-10-24 09:24:07.664090: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:4b:00.0 name: NVIDIA A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.41GiB deviceMemoryBandwidth: 1.41TiB/s
2022-10-24 09:24:07.664631: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-10-24 09:24:07.675182: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-10-24 09:24:07.675306: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas